In [ ]:
!pip install "tensorflow~=2.0"
!pip install "tensorflow-hub[make_image_classifier]~=0.6"

In [2]:
import argparse
import os
import time

import tensorflow as tf
import numpy as np
from PIL import Image

print(tf.__version__)

2.2.0


In [ ]:
!unzip ./dataset.zip -d ./dataset
!rm -r ./dataset.zip

In [4]:
# Very good, but can still be more accurate!
# DO NOT REMOVE THIS. I SPENT MULTIPLE HOURS SEARCHING FOR THE RIGHT ALGORITHM.
!make_image_classifier \
  --image_dir dataset \
  --batch_size 16 \
  --saved_model_dir anime-model \
  --tfhub_module https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 \
  --image_size 224 \
  --labels_output_file class_labels.txt \
  --tflite_output_file anime-tflite-model.tflite \
  --train_epochs 10

2020-07-20 10:37:45.070028: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
I0720 10:37:47.027791 140039463036800 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
I0720 10:37:47.028540 140039463036800 resolver.py:413] Downloading TF-Hub Module 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'.
I0720 10:37:47.382851 140039463036800 resolver.py:122] Downloaded https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4, Total size: 9.75MB
I0720 10:37:47.383447 140039463036800 resolver.py:428] Downloaded TF-Hub Module 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'.
2020-07-20 10:37:47.634436: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-20 10:37:47.695982: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [17]:
# Arguments to test the model.
ARGS_INPUT_MEAN = 127
IMAGES_TO_TEST = ['46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']

['46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']


In [14]:
# Load our label file.
def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]

In [21]:
def test_images(IMAGE_TO_TEST):
  for image_file in IMAGES_TO_TEST:
    # First, load the TFLite model that we just created and allocate some tensors.
    interpreter = tf.lite.Interpreter(model_path = "anime-tflite-model.tflite")
    interpreter.allocate_tensors()

    # Second, get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # check the type of the input tensor
    floating_model = input_details[0]['dtype'] == np.float32

    # NxHxWxC, H:1, W:2
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]
    img = Image.open(image_file).resize((width, height))

    # add N dim
    input_data = np.expand_dims(img, axis=0)

    if floating_model:
      input_data = (np.float32(input_data) - ARGS_INPUT_MEAN) / ARGS_INPUT_MEAN

    interpreter.set_tensor(input_details[0]['index'], input_data)

    start_time = time.time()
    interpreter.invoke()
    stop_time = time.time()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    results = np.squeeze(output_data)

    top_k = results.argsort()[-5:][::-1]
    labels = load_labels('class_labels.txt')
      
    for i in top_k:
      if floating_model:
        print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
      else:
        print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

    print('Time: {:.3f}ms'.format((stop_time - start_time) * 1000))
    print('\n')

test_images(IMAGE_TO_TEST)

0.817392: takimoto_hifumi
0.181480: sakurauchi_riko
0.001128: nishikino_maki
Time: 42.922ms


0.841065: takimoto_hifumi
0.140237: sakurauchi_riko
0.018698: nishikino_maki
Time: 36.472ms


0.936330: takimoto_hifumi
0.062487: sakurauchi_riko
0.001183: nishikino_maki
Time: 34.323ms


0.666609: sakurauchi_riko
0.331386: takimoto_hifumi
0.002005: nishikino_maki
Time: 35.564ms


0.873141: takimoto_hifumi
0.125340: sakurauchi_riko
0.001518: nishikino_maki
Time: 33.588ms


